In [1]:
# Scraper do Whoscored
# Alterar matchLink, roundNr e matchGame manualmente a cada jogo

import json
import numpy as np
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from mplsoccer import Pitch,VerticalPitch
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager
import math
import ast
from tqdm import tqdm

website = 'https://www.whoscored.com'
matchLink = "https://www.whoscored.com/Matches/1697398/Live/International-FIFA-World-Cup-2022-Portugal-Switzerland"
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path=r'/Users/menes/Documents/Data Hub/whoscored_scraper/chromedriver.exe',options=options)
driver.get(matchLink)
matchId = re.search('Matches/(\d+)/Live', matchLink)
matchId = matchId.group(1)
roundNr = 'WC'
matchGame = 'Portugal vs Switzerland'

soup = BeautifulSoup(driver.page_source,'lxml')
driver.quit()

data_cen = re.compile('var matchCentreData = ({.*?})')

data = soup.find_all("script")

event_data = str(data)

dicte = event_data.split("matchCentreData: ")
dicte = dicte[1].splitlines()
x = dicte[0][:-1]

matchdata = json.loads(x) 

x = pd.DataFrame.from_dict(matchdata["events"])
home = pd.DataFrame.from_dict(matchdata["home"],orient='index')
away = pd.DataFrame.from_dict(matchdata["away"],orient='index')


home[1] = away
print(home[0][0])
print(home[1][0])
home = home.T


C:\Users\menes\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\4258619027.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'/Users/menes/Documents/Data Hub/whoscored_scraper/chromedriver.exe',options=options)


340
423


In [2]:
# Organização geral dos dados
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
to_drop =  ['satisfiedEventsTypes']
x.drop(to_drop, inplace=True, axis=1)

# Whoscored (100x100) para metros (105x68)

"""
for row, id in enumerate(x.x):
    x.x[row] = x.x[row]*105/100
for row, id in enumerate(x.endX):
    x.endX[row] = x.endX[row]*105/100
for row, id in enumerate(x.blockedX):
    x.blockedX[row] = x.blockedX[row]*105/100
for row, id in enumerate(x.y):
    x.y[row] = x.y[row]*68/100
for row, id in enumerate(x.endY):
    x.endY[row] = x.endY[row]*68/100
for row, id in enumerate(x.blockedY):
    x.blockedY[row] = x.blockedY[row]*68/100

for row, id in enumerate(x.goalMouthZ):
    x.goalMouthZ[row] = x.goalMouthZ[row]*6.42/100
for row, id in enumerate(x.goalMouthY):
    x.goalMouthY[row] = x.goalMouthY[row]*68/100
"""

# Match Id e Field

x.insert(1, 'gameWeek', "")
x.insert(2, 'match', "")
x.insert(3, 'matchId', "")
x.insert(4, 'homeTeamId', "")
x.insert(5, 'awayTeamId', "")
x['gameWeek'] = x['gameWeek'].fillna(0).astype('string')
x['match'] = x['match'].fillna(0).astype('string')

for row, id in enumerate(x.gameWeek):
    x.gameWeek[row] = roundNr
for row, id in enumerate(x.match):
    x.match[row] = matchGame
for row, id in enumerate(x.matchId):
    x.matchId[row] = matchId

x.insert(len(x.keys()), 'timestamp', "")
for row, id in enumerate(x.timestamp):
    x.timestamp[row] = x.minute[row]*60+x.second[row]
x['timestamp'] = x['timestamp'].fillna(0).astype('int')

# Organização geral dos dados

x['playerId'] = x['playerId'].fillna(0).astype('int')
x['second'] = x['second'].fillna(0).astype('int')
x['period'] = x['period'].fillna(0).astype('string')
x['type'] = x['type'].fillna(0).astype('string')
x['matchId'] = x['matchId'].fillna(0).astype('int')
x['outcomeType'] = x['outcomeType'].fillna(0).astype('string')
x['qualifiers'] = x['qualifiers'].fillna(0).astype('string')
x['relatedEventId'] = x['relatedEventId'].fillna(0).astype('int')
x['relatedPlayerId'] = x['relatedPlayerId'].fillna(0).astype('int')


x.period = x['period'].replace("{'value': 16, 'displayName': 'PreMatch'}", 'Pre Match')
x.period = x['period'].replace("{'value': 1, 'displayName': 'FirstHalf'}", '1st Half')
x.period = x['period'].replace("{'value': 2, 'displayName': 'SecondHalf'}", '2nd Half')
x.period = x['period'].replace("{'value': 3, 'displayName': 'FirstPeriodOfExtraTime'}", '1st Half - Extra Time')
x.period = x['period'].replace("{'value': 4, 'displayName': 'SecondPeriodOfExtraTime'}", '2nd Half - Extra Time')
x.period = x['period'].replace("{'value': 5, 'displayName': 'PenaltyShootout'}", 'Penalty Shootout')
x.period = x['period'].replace("{'value': 14, 'displayName': 'PostGame'}", 'Post Game')
x.outcomeType = x['outcomeType'].replace("{'value': 1, 'displayName': 'Successful'}", 'Successful')
x.outcomeType = x['outcomeType'].replace("{'value': 0, 'displayName': 'Unsuccessful'}", 'Unsuccessful')
x.type = x['type'].replace("{'value': 32, 'displayName': 'Start'}", 'Start')
x.type = x['type'].replace("{'value': 1, 'displayName': 'Pass'}", 'Pass')
x.type = x['type'].replace("{'value': 44, 'displayName': 'Aerial'}", 'Aerial Duel')
x.type = x['type'].replace("{'value': 50, 'displayName': 'Dispossessed'}", 'Dispossessed')
x.type = x['type'].replace("{'value': 7, 'displayName': 'Tackle'}", 'Tackle')
x.type = x['type'].replace("{'value': 13, 'displayName': 'MissedShots'}", 'Missed Shot')
x.type = x['type'].replace("{'value': 30, 'displayName': 'End'}", 'End')
x.type = x['type'].replace("{'value': 34, 'displayName': 'FormationSet'}", 'Formation Set')
x.type = x['type'].replace("{'value': 8, 'displayName': 'Interception'}", 'Interception')
x.type = x['type'].replace("{'value': 49, 'displayName': 'BallRecovery'}", 'Ball Recovery')
x.type = x['type'].replace("{'value': 12, 'displayName': 'Clearance'}", 'Clearance')
x.type = x['type'].replace("{'value': 6, 'displayName': 'CornerAwarded'}", 'Corner Awarded')
x.type = x['type'].replace("{'value': 61, 'displayName': 'BallTouch'}", 'Ball Touch')
x.type = x['type'].replace("{'value': 74, 'displayName': 'BlockedPass'}", 'Blocked Pass')
x.type = x['type'].replace("{'value': 45, 'displayName': 'Challenge'}", 'Dribbled')
x.type = x['type'].replace("{'value': 3, 'displayName': 'TakeOn'}", 'Dribble')
x.type = x['type'].replace("{'value': 42, 'displayName': 'GoodSkill'}", 'Good Skill')
x.type = x['type'].replace("{'value': 52, 'displayName': 'KeeperPickup'}", 'Keeper Pickup')
x.type = x['type'].replace("{'value': 4, 'displayName': 'Foul'}", 'Foul')
x.type = x['type'].replace("{'value': 10000, 'displayName': 'OffsideGiven'}", 'Offside Given')
x.type = x['type'].replace("{'value': 2, 'displayName': 'OffsidePass'}", 'Offside Pass')
x.type = x['type'].replace("{'value': 55, 'displayName': 'OffsideProvoked'}", 'Offside Provoked')
x.type = x['type'].replace("{'value': 16, 'displayName': 'Goal'}", 'Goal')
x.type = x['type'].replace("{'value': 11, 'displayName': 'Claim'}", 'Claim')
x.type = x['type'].replace("{'value': 18, 'displayName': 'SubstitutionOff'}", 'Substitution Off')
x.type = x['type'].replace("{'value': 19, 'displayName': 'SubstitutionOn'}", 'Substitution On')
x.type = x['type'].replace("{'value': 40, 'displayName': 'FormationChange'}", 'Formation Change')
x.type = x['type'].replace("{'value': 17, 'displayName': 'Card'}", 'Card')
x.type = x['type'].replace("{'value': 59, 'displayName': 'KeeperSweeper'}", 'Keeper Sweeper')
x.type = x['type'].replace("{'value': 58, 'displayName': 'PenaltyFaced'}", 'Penalty Faced')
x.type = x['type'].replace("{'value': 15, 'displayName': 'SavedShot'}", 'Saved Shot')
x.type = x['type'].replace("{'value': 10, 'displayName': 'Save'}", 'Save')
x.type = x['type'].replace("{'value': 51, 'displayName': 'Error'}", 'Error')
x.type = x['type'].replace("{'value': 56, 'displayName': 'ShieldBallOpp'}", 'Shield Ball OOP')
x.type = x['type'].replace("{'value': 41, 'displayName': 'Punch'}", 'Punch')
x.type = x['type'].replace("{'value': 54, 'displayName': 'Smother'}", 'Smother')
x.type = x['type'].replace("{'value': 14, 'displayName': 'ShotOnPost'}", 'Shot On Post')
x.type = x['type'].replace("{'value': 60, 'displayName': 'ChanceMissed'}", 'Chance Missed')
x.cardType = x['cardType'].replace("{'value': 31, 'displayName': 'Yellow'}", 'Yellow Card')
x.cardType = x['cardType'].replace("{'value': 33, 'displayName': 'Red'}", 'Red Card')
x.type = x['type'].replace("{'value': 53, 'displayName': 'CrossNotClaimed'}", 'Cross Not Claimed')



C:\Users\menes\AppData\Local\Temp\ipykernel_16932\3115313978.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.matchId[row] = matchId
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\3115313978.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.timestamp[row] = x.minute[row]*60+x.second[row]


In [3]:
# Organização geral dos dados

for index, type in enumerate(x.type):
    if (type == 'Dribbled'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Goal'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Shot On Post'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Missed Shot'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Save'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Offside Pass'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Clearance'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Interception'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Dispossessed'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Tackle') and (x.outcomeType[index] == 'Successful'):
        x.type[index]='Complete Tackle'
for index, type in enumerate(x.type):
    if (type == 'Tackle'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Complete Tackle'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel') and (x.outcomeType[index] == 'Unsuccessful'):
        x.type[index]='Aerial Duel Lost'
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel') and (x.outcomeType[index] == 'Successful'):
        x.type[index]='Aerial Duel Won'
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel Lost'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel Won'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Corner Awarded') and (x.outcomeType[index] == 'Unsuccessful'):
        x.type[index]='Corner Conceded'
for index, type in enumerate(x.type):
    if (type == 'Corner Awarded'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Corner Conceded'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Foul') and (x.outcomeType[index] == 'Successful'):
        x.type[index]='Fouled'
for index, type in enumerate(x.type):
    if (type == 'Foul'):
        x.outcomeType[index]=''
for index, type in enumerate(x.type):
    if (type == 'Fouled'):
        x.outcomeType[index]=''
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('OutfielderBlock') != -1):
        x.type[index] = 'Block'
for row, type in enumerate(x.type):
    if (type == 'Block'):
        if (x.type[row-1] == 'Saved Shot'):
            x.type[row-1] = 'Blocked Shot'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('LeadingToAttempt') != -1):
        x.type[index]='Error Leading To Attempt'
    if (qualifier.find('LeadingToGoal') != -1):
        x.type[index]='Error Leading To Goal'
x.insert(len(x.keys()), 'isGoodSkill', "")
for row, type in enumerate(x.type):
    if (type == 'Good Skill'):
        if (x.type[row-1] == 'Dribble') and (x.outcomeType[row-1] == 'Successful'):
            x.isGoodSkill[row-1] = 'True'

In [4]:
# Organização geral dos dados

x.insert(len(x.keys()), 'bodyPart', "")
x.insert(len(x.keys()), 'situation', "")
x.insert(len(x.keys()), 'passType', "")
x.insert(len(x.keys()), 'isChipped', "")
x.insert(len(x.keys()), 'isBigChance', "")
x.insert(len(x.keys()), 'isKeyPass', "")
x.insert(len(x.keys()), 'isAssist', "")
x.insert(len(x.keys()), 'isLongBall', "")
x.insert(len(x.keys()), 'is2ndAssist', "")
x['bodyPart'] = x['bodyPart'].fillna(0).astype('string')
x['situation'] = x['situation'].fillna(0).astype('string')
x['passType'] = x['passType'].fillna(0).astype('string')
x['isKeyPass'] = x['isKeyPass'].fillna(0).astype('bool')
x['isBigChance'] = x['isBigChance'].fillna(0).astype('bool')
x['isAssist'] = x['isAssist'].fillna(0).astype('bool')
x['is2ndAssist'] = x['is2ndAssist'].fillna(0).astype('bool')
x['isGoal'] = x['isGoal'].fillna(0).astype('bool')
for row, type in enumerate(x.type):
    if (type == 'Goal'):
        if (x.type[row-1] == 'Pass') and (x.outcomeType[row-1] == 'Successful'):
            x.isAssist[row-1] = 'True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('IntentionalGoalAssist') != -1):
        x.isAssist[index] = 'True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('Chipped') != -1):
        x.isChipped[index] = 'True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('Longball') != -1):
        x.isLongBall[index] = 'True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('RightFoot') != -1):
        x.bodyPart[index]='Right Foot'
    if (qualifier.find('LeftFoot') != -1):
        x.bodyPart[index]='Left Foot'
    if (qualifier.find('Head') != -1):
        x.bodyPart[index]='Head'
    if (qualifier.find('OtherBodyPart') != -1):
        x.bodyPart[index]='Other Body Part'
    if (qualifier.find('Hands') != -1):
        x.bodyPart[index]='Hands'
    if (qualifier.find('Feet') != -1):
        x.bodyPart[index]='Feet'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('Cross') != -1):
        x.passType[index]='Cross'
    if (qualifier.find('Throughball') != -1):
        x.passType[index]='Through Ball'
    if (qualifier.find('Headpass') != -1):
        x.passType[index]='Head Pass'
    if (qualifier.find('CornerTaken') != -1):
        x.situation[index]='Corner'
    if (qualifier.find('FreekickTaken') != -1):
        x.situation[index]='Freekick'
    if (qualifier.find('ThrowIn') != -1):
        x.situation[index]='Throw In'
    if (qualifier.find('KeeperThrow') != -1):
        x.situation[index]='Keeper Throw'
for index, type in enumerate(x.type):
    if (type == 'Pass') and ((x.endY[index]-x.y[index]) > 40) :
        x.passType[index]='Switch Of Play'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('BigChance') != -1):
        x.isBigChance[index]='True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('KeyPass') != -1):
        x.isKeyPass[index]='True'
for index, qualifier in enumerate(x.qualifiers):
    if (qualifier.find('RegularPlay') != -1):
        x.situation[index]='Regular Play'
    if (qualifier.find('FromCorner') != -1):
        x.situation[index]='From Corner'
    if (qualifier.find('DirectFreekick') != -1):
        x.situation[index]='Direct Freekick'
    if (qualifier.find('FastBreak') != -1):
        x.situation[index]='Fast Break'
    if (qualifier.find('FastBreak') != -1):
        x.situation[index]='Fast Break'
    if (qualifier.find('GoalKick') != -1):
        x.situation[index]='Goal Kick' 
    if (qualifier.find('IndirectFreekickTaken') != -1):
        x.situation[index]='Indirect Freekick'
    if (qualifier.find('SetPiece') != -1):
        x.situation[index]='Set Piece'
    if (qualifier.find('ThrowinSetPiece') != -1):
        x.situation[index]='From Throw In Set Piece'
    if (qualifier.find('Penalty') != -1):
        x.situation[index]='Penalty'   
for row, isAssist in enumerate(x.isAssist):
    if (isAssist == 'True'):
        if (x.type[row-1] == 'Pass') and (x.outcomeType[row-1] == 'Successful'):
            x.is2ndAssist[row-1] = 'True'
for index, type in enumerate(x.type):
    if (type == 'Aerial Duel Won'):
        x.bodyPart[index]='Head'

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2675537827.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.isAssist[row-1] = 'True'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2675537827.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.isChipped[index] = 'True'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2675537827.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.isLongBall[index] = 'True'
C:\Users\mene

In [5]:
# Passador e Receptor

x.insert(len(x.keys()), 'receivedFrom', "")
x.insert(len(x.keys()), 'deliveredTo', "")
for row, type in enumerate(x.type):
    if (type == 'Pass') and (x.outcomeType[row] == 'Successful'):
        x.receivedFrom[row+1] = x.playerId[row]
for row, type in enumerate(x.type):
    if (type == 'Pass') and (x.outcomeType[row] == 'Successful'):
        x.deliveredTo[row] = x.playerId[row+1]

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2570143535.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.receivedFrom[row+1] = x.playerId[row]
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2570143535.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.deliveredTo[row] = x.playerId[row+1]


In [6]:
# Carries

min_dribble_length: float = 3.0
max_dribble_length: float = 105.0
max_dribble_duration: float = 20.0
next_actions = x.shift(-1)
same_team = x.teamId == next_actions.teamId
dx = x.endX - next_actions.x
dy = x.endY- next_actions.y
far_enough=dy** 2 + dy** 2 >= min_dribble_length** 2
not_too_far = dx ** 2+ dy** 2<= max_dribble_length ** 2
dt = next_actions.timestamp - x.timestamp
same_phase = dt < max_dribble_duration
same_period = x.period == next_actions.period
dribble_idx = same_team & far_enough & not_too_far & same_phase & same_period
dribbles = pd.DataFrame()
prev = x[dribble_idx]
nex= next_actions[dribble_idx]
dribbles['matchId' ] = nex.matchId
dribbles['period' ] = nex.period
morecols = ['playerId']
for cols in ['expandedMinute']:
	dribbles[cols] = nex[cols]
for cols in morecols:
		dribbles[cols] = nex[cols]
dribbles['eventId'] = (prev.eventId + 0.1)
dribbles['id'] = (prev.id + 0.1)
dribbles['matchId'] = x['matchId']
dribbles['match'] = x['match']
dribbles['gameWeek'] = x['gameWeek']
dribbles['minute' ] = (prev.minute + nex.minute) / 2
dribbles['second' ] = (prev.second + nex.second) / 2
dribbles['timestamp' ] = (prev.timestamp + nex.timestamp) / 2
dribbles['teamId'] = nex.teamId.astype(int)
dribbles['x'] = prev.endX
dribbles['y']= prev.endY
dribbles['endX'] = nex.x
dribbles['endY'] = nex.y
dribbles['type'] = ['Carry' for _ in range(len(dribbles))]
dribbles['outcomeType'] = ['Successful' for _ in range(len(dribbles))]
dribbles['playerId'] = dribbles['playerId'].fillna(0).astype('int')

In [7]:
dribbles.insert(16,'distance', "")
dribbles['distance'] = dribbles['distance'].fillna(0).astype('bool')
dribbles['distance'] = np.sqrt(np.square(dribbles.endX - dribbles.x) + np.square(dribbles.endY - dribbles.y))

In [8]:
dribbles = dribbles[(dribbles['type'] == 'Carry') & (dribbles['distance'] >= 3)]

In [9]:
# Merge Carries

x=pd.concat([x,dribbles], join='outer',axis=0)


In [10]:
from pandas.api.types import CategoricalDtype
period_order = CategoricalDtype(
    ['Pre Match', '1st Half', '2nd Half','1st Half - Extra Time', '2nd Half - Extra Time', 'Penalty Shootout','Post Game'], 
    ordered=True
)
x.period=x.period.astype(period_order)
x=x.sort_values(['period','minute','second','timestamp','id','eventId',]).reset_index(drop=True)
x['eventId']=range(len(x))

In [11]:
x.insert(len(x.keys()), 'leadingTo', "")
for row, type in enumerate(x.type):
    if (type == 'Carry'):
        if (x.isShot[row+1] == 'True'):
            x.leadingTo[row] = 'Shot'

In [12]:
for row, type in enumerate(x.type):
    if (type == 'Carry'):
        if (x.isKeyPass[row+1] == 'True'):
            x.leadingTo[row] = 'KeyPass'

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1683290032.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.leadingTo[row] = 'KeyPass'


In [13]:
conceded = x.loc[(x['type']=='Goal')]

In [14]:
conceded.id= conceded.id +0.1

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1219493462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceded.id= conceded.id +0.1


In [15]:
conceded.second= conceded.second +0.1
conceded.expandedMinute= conceded.expandedMinute +0.1
conceded.eventId= conceded.eventId +0.1
conceded.timestamp= conceded.timestamp +0.1
conceded.type = 'Goal Conceded'
conceded.bodyPart = ''
conceded.isKeyPass = ''
conceded.isAssist = ''
conceded.isLongBall = ''
conceded.is2ndAssist = ''
conceded.receivedFrom = ''
conceded.cardType = ''
conceded.relatedPlayerId = ''
conceded.relatedEventId = ''
conceded.qualifiers = ''
conceded.teamId = ''
conceded.playerId = ''
conceded.isTouch = 'False'

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1568443585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceded.second= conceded.second +0.1
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1568443585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conceded.expandedMinute= conceded.expandedMinute +0.1
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1568443585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [16]:
x=pd.concat([x,conceded], join='outer',axis=0)

In [17]:
x=x.sort_values(['period','minute','second','timestamp','id','eventId',]).reset_index(drop=True)

In [18]:
# Organização home/away Id

temp_x = home.drop_duplicates(subset='teamId')

home_tmp = temp_x.drop(['scores', 'players', 'stats', 'formations', 'incidentEvents', 'name', 'countryName', 'managerName', 'shotZones', 'averageAge'], axis=1)
x = x.merge(home_tmp, left_on=['teamId'], right_on=['teamId'], how='left', suffixes=['x', 'home'])
x_columns = [col for col in x.columns if col != 'field']
x_columns.insert(6, 'field')
x = x[x_columns]
for row, field in enumerate(x.field):
    if field == 'home':
        x.homeTeamId = x.teamId[row]
    elif field == 'away':
        x.awayTeamId = x.teamId[row]

In [19]:
for row, type in enumerate(x.type):
    if (type == 'Goal Conceded'):
        x.teamId[row] = x.teamId[row+1]
for row, type in enumerate(x.type):
    if (type == 'Goal Conceded'):
        x.field[row] = x.field[row+1]

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\105866043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.teamId[row] = x.teamId[row+1]
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\105866043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.field[row] = x.field[row+1]


In [ ]:
#xT

x.insert(len(x.keys()), 'isProgressive', "")
y = x.loc[(x['type']=='Pass') & (x['outcomeType'] == 'Successful') | (x['type']=='Carry') & (x['outcomeType'] == 'Successful')]
xt = pd.read_excel ('xt_grid.xlsx',header =None)
xt =np.array (xt)
xt_rows, xt_cols = xt.shape
y ['x1_bin'] = pd.cut (y['x'], bins = xt_cols, labels=False)
y ['y1_bin'] = pd.cut (y['y'], bins = xt_rows, labels=False)
y ['x2_bin'] = pd.cut (y['endX'], bins = xt_cols, labels=False)
y ['y2_bin'] = pd.cut (y['endY'], bins = xt_rows, labels=False)
y['start_zone_value'] = y [['x1_bin', 'y1_bin']].apply(lambda x: xt[x[1]][x[0]], axis=1)
y['x2_bin'] = y['x2_bin'].apply(lambda x: int(x) if x == x else 0)
y['y2_bin'] = y['y2_bin'].apply(lambda x: int(x) if x == x else 0)
y['end_zone_value'] = y [['x2_bin', 'y2_bin']].apply(lambda x: xt[x[1]][x[0]], axis=1)
y['x2_bin'] = y['x2_bin'].apply(lambda x: int(x) if x == x else 0)
y['y2_bin'] = y['y2_bin'].apply(lambda x: int(x) if x == x else 0)
y['end_zone_value'] = y [['x2_bin', 'y2_bin']].apply(lambda x: xt[x[1]][x[0]], axis=1)
y['xT'] = y['end_zone_value'] - y['start_zone_value']
z = y.drop_duplicates(subset='id')
y=z.drop(['eventId', 'match','gameWeek','homeTeamId', 'awayTeamId','isProgressive', 'distance','isLongBall','isGoodSkill','matchId', 'field', 'timestamp', 'receivedFrom', 'deliveredTo','minute', 'second', 'teamId','x','y','expandedMinute','period','type','outcomeType','qualifiers','isTouch','playerId','endX','endY','relatedEventId','relatedPlayerId','blockedX','blockedY','goalMouthZ','goalMouthY','isShot','cardType','leadingTo','is2ndAssist','isAssist', 'isGoal','isBigChance','isKeyPass','situation','isChipped','passType','bodyPart','x1_bin','y1_bin','x2_bin','y2_bin','start_zone_value','end_zone_value'], axis=1)
x= x.merge(y, left_on=['id'], right_on=['id'], how='left', suffixes=['x','x'])

In [ ]:
#EPV

yy = x.loc[(x['type']=='Pass') & (x['outcomeType'] == 'Successful') | (x['type']=='Carry') & (x['outcomeType'] == 'Successful')]
epv = pd.read_excel ('EPV_grid.xlsx',header =None)
epv =np.array (epv)
epv_rows, epv_cols = epv.shape
yy['x1_bin'] = pd.cut (yy['x'], bins = epv_cols, labels=False)
yy['y1_bin'] = pd.cut (yy['y'], bins = epv_rows, labels=False)
yy['x2_bin'] = pd.cut (yy['endX'], bins = epv_cols, labels=False)
yy['y2_bin'] = pd.cut (yy['endY'], bins = epv_rows, labels=False)
yy['start_zone_value'] = yy[['x1_bin', 'y1_bin']].apply(lambda x: epv[x[1]][x[0]], axis=1)
yy['x2_bin'] = yy['x2_bin'].apply(lambda x: int(x) if x == x else 0)
yy['y2_bin'] = yy['y2_bin'].apply(lambda x: int(x) if x == x else 0)
yy['end_zone_value'] = yy[['x2_bin', 'y2_bin']].apply(lambda x: epv[x[1]][x[0]], axis=1)
yy['x2_bin'] = yy['x2_bin'].apply(lambda x: int(x) if x == x else 0)
yy['y2_bin'] = yy['y2_bin'].apply(lambda x: int(x) if x == x else 0)
yy['end_zone_value'] = yy[['x2_bin', 'y2_bin']].apply(lambda x: epv[x[1]][x[0]], axis=1)
yy['start_zone_value'] = pd.to_numeric(yy['start_zone_value'],errors='coerce')
yy['end_zone_value'] = pd.to_numeric(yy['end_zone_value'],errors='coerce')
yy['EPV'] = yy['end_zone_value'] - yy['start_zone_value']
z1 = yy.drop_duplicates(subset='id')
yy=z1.drop(['eventId', 'match','gameWeek','homeTeamId', 'awayTeamId','isProgressive', 'xT','distance','isLongBall','isGoodSkill','matchId', 'field', 'timestamp', 'receivedFrom', 'deliveredTo','minute', 'second', 'teamId','x','y','expandedMinute','period','type','outcomeType','qualifiers','isTouch','playerId','endX','endY','relatedEventId','relatedPlayerId','blockedX','blockedY','goalMouthZ','goalMouthY','isShot','cardType','leadingTo','is2ndAssist','isAssist', 'isGoal','isBigChance','isKeyPass','situation','isChipped','passType','bodyPart','x1_bin','y1_bin','x2_bin','y2_bin','start_zone_value','end_zone_value'], axis=1)
x= x.merge(yy, left_on=['id'], right_on=['id'], how='left', suffixes=['x','x'])


In [20]:
# Progressivo
def isProgressive(x,y,endX,endY):
    distanceInitial = np.sqrt(np.square(105 - x) + np.square(68 - y))
    distanceFinal = np.sqrt(np.square(105 - endX) + np.square(68 - endY))
    if(x <= 52.5 and endX <= 52.5):
        if(distanceInitial - distanceFinal >= 30) and (endX-x >= 5):
            return 'True'
    elif(x <= 52.5 and endX > 52.5):
        if(distanceInitial - distanceFinal >= 15) and (endX-x >= 5):
            return 'True'
    elif(x > 52.5 and endX > 52.5):
        if(distanceInitial - distanceFinal >= 10) and (endX-x >= 5):
            return 'True'
    return 'False'

for index, progressive in enumerate(x.isProgressive):
    if x.type[index]=='Pass':
        x.isProgressive[index]= isProgressive(x.x[index],x.y[index],x.endX[index], x.endY[index])
for index, progressive in enumerate(x.isProgressive):
    if x.type[index]=='Carry':
        x.isProgressive[index]= isProgressive(x.x[index],x.y[index],x.endX[index], x.endY[index])

AttributeError: 'DataFrame' object has no attribute 'isProgressive'

In [ ]:
# xG e xGOT do Fotmob (excel extraído pelo R)

xG = pd.read_excel ('shotsTotal.xlsx')
xG2 = xG.drop_duplicates(subset='id')
xG=xG2.drop(['event_type','team_id', 'player_id','player_name', 'x','y', 'min','min_added','blocked_x','blocked_y','goal_crossed_y','goal_crossed_z','shot_type','situation','period','on_goal_shot','first_name','last_name','team_color'], axis=1)
x= x.merge(xG, left_on=['id'], right_on=['id'], how='left', suffixes=['x','x'])
x_columns = [col for col in x.columns if col != 'is_own_goal']
x_columns.insert(31, 'is_own_goal')
x = x[x_columns]
x.rename(columns={'is_own_goal': 'isOwnGoal'}, inplace=True)
x.rename(columns={'expected_goals': 'xG'}, inplace=True)
x.rename(columns={'expected_goals_on_target': 'xGOT'}, inplace=True)
x.rename(columns={'is_blocked': 'isBlocked'}, inplace=True)
x.rename(columns={'is_on_target': 'isOnTarget'}, inplace=True)

In [ ]:
# xSave remates à baliza

x.insert(len(x.keys()), 'xSave', "")
for row, type in enumerate(x.type):
    if (type == 'Save'):
        if (x.type[row-1] == 'Saved Shot'):
            x.xSave[row] = 1- x.xGOT[row-1]

In [ ]:
for row, type in enumerate(x.type):   
    if (type == 'Goal Conceded'):
        if (x.type[row-1] == 'Goal'):
            x.xSave[row] = 1- x.xGOT[row-1]

In [ ]:
for row, type in enumerate(x.type):
    if (type == 'Save'):
        if (x.type[row-1] == 'Saved Shot'):
            x.url[row] = x.url[row-1]

In [ ]:
for row, type in enumerate(x.type):   
    if (type == 'Goal Conceded'):
        if (x.type[row-1] == 'Goal'):
            x.url[row] = x.url[row-1]

In [ ]:
# mais coordenadas
for index, row in x.iterrows():
 if row['goalMouthY'] > 0.0:
  x.loc[index, 'endX'] = 105
  x.loc[index, 'endY'] = row['goalMouthY']

In [ ]:
# mais coordenadas
for index, row in x.iterrows():
 if row['blockedX'] > 0.0:
  x.loc[index, 'endX'] = row['blockedX']
for index, row in x.iterrows():
 if row['blockedY'] > 0.0:
  x.loc[index, 'endY'] = row['blockedY']

In [ ]:
# Tableau Path
x.insert(45, 'path', '')
for index, type in enumerate(x.type):
    if (type == 'Pass') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Carry') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Clearance') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Goal') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Missed Shot') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Shot On Post') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Saved Shot') :
        x.path[index]='1'
for index, type in enumerate(x.type):
    if (type == 'Blocked Shot') :
        x.path[index]='1'

In [23]:
# Distâncias

x['distance'] = np.sqrt(np.square(x.endX - x.x) + np.square(x.endY - x.y))

In [24]:
# Organização de dados

x = x.replace(np.nan, '', regex=True)
x_bodyPart=x[(x['type'] == 'Pass') & (x['bodyPart'] == '') & (x['situation']!='Throw In') & (x['situation']!='Keeper Throw')]
x_bodyPart['bodyPart'] = 'Feet'
x.update(x_bodyPart)


C:\Users\menes\AppData\Local\Temp\ipykernel_16932\917012384.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_bodyPart['bodyPart'] = 'Feet'


In [25]:
x_clearPart=x[(x['type'] == 'Clearance') & (x['bodyPart'] != 'Head')]
x_clearPart['bodyPart'] = 'Feet'
x.update(x_clearPart)

x_carryPart=x[(x['type'] == 'Carry')]
x_carryPart['bodyPart'] = 'Feet'
x_carryPart['isTouch'] = 'True'
x.update(x_carryPart)

x['teamId'] = x['teamId'].astype(int)
x['homeTeamId'] = x['homeTeamId'].astype(int)
x['awayTeamId'] = x['awayTeamId'].astype(int)
x['matchId'] = x['matchId'].astype(int)
x['minute'] = x['minute'].astype(int)
x['second'] = x['second'].astype(int)
x['expandedMinute'] = x['expandedMinute'].astype(int)
x['timestamp'] = x['timestamp'].astype(int)

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2539891570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_clearPart['bodyPart'] = 'Feet'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2539891570.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_carryPart['bodyPart'] = 'Feet'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2539891570.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [26]:
# GameState and Score

x['homeTeamScore'] = 0
x['awayTeamScore'] = 0

x['homeTeamScore'] = x['homeTeamScore'].astype(int)
x['awayTeamScore'] = x['awayTeamScore'].astype(int)

for i in range(1, len(x)):
 if x.loc[i, "type"] == "Goal" and x.loc[i, "field"] == "home" and x.loc[i, "isGoal"] == True:
  x.loc[i+2:, "homeTeamScore"] += 1
 elif x.loc[i, "type"] == "Goal" and x.loc[i, "field"] == "away" and x.loc[i, "isGoal"] == True:
  x.loc[i+2:, "awayTeamScore"] += 1
 elif x.loc[i, "type"] == "Goal" and x.loc[i, "field"] == "away" and x.loc[i, "isOwnGoal"] == True:
  x.loc[i+1:, "homeTeamScore"] += 1
 elif x.loc[i, "type"] == "Goal" and x.loc[i, "field"] == "home" and x.loc[i, "isOwnGoal"] == True:
  x.loc[i+1:, "awayTeamScore"] += 1 

x.insert(len(x.keys()), 'gameState', "")

for index, field in enumerate(x.field):
 if (field == 'home') and ((x.homeTeamScore[index] - x.awayTeamScore[index]) == 1):
  x.gameState[index] = 'Winning by 1'
 elif (field == 'home') and ((x.homeTeamScore[index] - x.awayTeamScore[index]) > 1):
  x.gameState[index] = 'Winning by +1'
 elif (field == 'home') and ((x.homeTeamScore[index] - x.awayTeamScore[index]) == -1):
  x.gameState[index] = 'Losing by 1'
 elif (field == 'home') and ((x.homeTeamScore[index] - x.awayTeamScore[index]) <-1):
  x.gameState[index] = 'Losing by +1'
 elif (field == 'away') and ((x.awayTeamScore[index] - x.homeTeamScore[index]) ==-1):
  x.gameState[index] = 'Losing by 1'
 elif (field == 'away') and ((x.awayTeamScore[index] - x.homeTeamScore[index]) <-1):
  x.gameState[index] = 'Losing by +1'  
 elif (field == 'away') and ((x.awayTeamScore[index] - x.homeTeamScore[index]) == 1):
  x.gameState[index] = 'Winning by 1'
 elif (field == 'away') and ((x.awayTeamScore[index] - x.homeTeamScore[index]) > 1):
  x.gameState[index] = 'Winning by +1'
 elif (field == 'home') and ((x.homeTeamScore[index] - x.awayTeamScore[index]) == 0):
  x.gameState[index] = 'Drawing'
 elif (field == 'away') and ((x.awayTeamScore[index] - x.homeTeamScore[index]) == 0):
  x.gameState[index] = 'Drawing'

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\249068482.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.gameState[index] = 'Drawing'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\249068482.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.gameState[index] = 'Drawing'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\249068482.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.gameState[index] = 'Losing by 1'
C:\

In [27]:
# Organização das colunas
#x.insert(len(x.keys()), 'position', "")
x = x[['id', 'gameWeek', 'match', 'matchId', 'homeTeamId','awayTeamId','eventId','playerId', 'position','teamId','field','period','homeTeamScore','awayTeamScore','gameState','minute','second','expandedMinute','timestamp','type','outcomeType','isTouch','qualifiers','cardType','bodyPart','situation','x','y','endX','endY','distance','blockedX','blockedY','goalMouthY', 'goalMouthZ','passType','deliveredTo','receivedFrom','leadingTo','isChipped','isLongBall','isBigChance','is2ndAssist','isKeyPass','isAssist','isProgressive','isGoodSkill','isShot','isGoal','path']]

KeyError: "['position', 'isProgressive', 'path'] not in index"

In [28]:
home = home[['teamId', 'field', 'formations']]

In [29]:
home = home.set_index(['teamId'],['field'],['formations']).apply(pd.Series.explode).reset_index()

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1322391633.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  home = home.set_index(['teamId'],['field'],['formations']).apply(pd.Series.explode).reset_index()


In [31]:
# Posições
home2=pd.DataFrame(home["formations"].to_list(), columns=['formationId','formationName','captainPlayerId','period','startMinuteExpanded','endMinuteExpanded','jerseyNumbers','formationSlots','playerIds','formationPositions'])
home3 = pd.concat([home, home2], axis=1)
home = home3[['teamId','field','formationId','formationName','period','startMinuteExpanded','endMinuteExpanded','jerseyNumbers','playerIds','formationSlots']]
homex= home[['playerIds']]
homexx= home[['formationSlots']]
homexxx= home[['jerseyNumbers']]
homex =homex.explode('playerIds')
homexx = homexx.explode('formationSlots')
homexxx = homexxx.explode('jerseyNumbers')
home = home[['teamId','field','formationId','formationName','period','startMinuteExpanded','endMinuteExpanded']]
home = pd.concat([home, homex,homexx,homexxx], axis=1)
formations = home[['teamId','field','formationId','formationName','period','startMinuteExpanded','endMinuteExpanded','jerseyNumbers','playerIds','formationSlots']]
formations.insert(0, 'matchId', "")
formations['matchId'] = x['matchId']
formations.insert(7, 'startTimestamp', "")
formations['startTimestamp'] = formations['startMinuteExpanded']*60
formations.insert(9, 'endTimestamp', "")
formations['endTimestamp'] = formations['endMinuteExpanded']*60
formations.insert(13, 'position', "")
formations = formations.reset_index()
formations.rename(columns = {'playerIds':'playerId'}, inplace = True)

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1948272401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formations['matchId'] = x['matchId']
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1948272401.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formations['startTimestamp'] = formations['startMinuteExpanded']*60
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\1948272401.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [32]:
formations

,index,matchId,teamId,field,formationId,formationName,period,startMinuteExpanded,startTimestamp,endMinuteExpanded,endTimestamp,jerseyNumbers,playerId,formationSlots,position
0,0,1697398,340,home,4,433,16,0,0,76,4560,22,373842,1,
1,0,1697398,340,home,4,433,16,0,0,76,4560,2,353418,2,
2,0,1697398,340,home,4,433,16,0,0,76,4560,5,115375,3,
3,0,1697398,340,home,4,433,16,0,0,76,4560,14,99911,4,
4,0,1697398,340,home,4,433,16,0,0,76,4560,3,10105,5,
5,0,1697398,340,home,4,433,16,0,0,76,4560,4,313171,6,
6,0,1697398,340,home,4,433,16,0,0,76,4560,25,120608,7,
7,0,1697398,340,home,4,433,16,0,0,76,4560,10,136741,8,
8,0,1697398,340,home,4,433,16,0,0,76,4560,26,395802,9,
9,0,1697398,340,home,4,433,16,0,0,76,4560,8,123761,10,


In [33]:
# Posições tática

for index, formationId in enumerate(formations.formationId):
 if formations.formationSlots[index] == 0:
  formations.position[index] = 'Bench'
 elif formations.formationSlots[index] == 1:
  formations.position[index] = 'GK'
 elif (formationId == 2) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 2) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 2) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RCM'
 elif (formationId == 2) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 2) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 2) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 2) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 2) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 2) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 2) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 3) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 3) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 3) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 3) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 3) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 3) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 3) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'CAM'
 elif (formationId == 3) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 3) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 3) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCM'
 elif (formationId == 4) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 4) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 4) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 4) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 4) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 4) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 4) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 4) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 4) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RW'
 elif (formationId == 4) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LW'
 elif (formationId == 5) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 5) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 5) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RDM'
 elif (formationId == 5) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 5) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 5) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 5) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LDM'
 elif (formationId == 5) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 5) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 5) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 6) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 6) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 6) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RCM'
 elif (formationId == 6) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 6) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 6) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 6) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 6) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 6) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 6) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 7) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 7) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 7) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 7) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 7) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 7) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 7) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 7) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 7) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'LCM'
 elif (formationId == 7) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 8) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 8) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 8) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LDM'
 elif (formationId == 8) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 8) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 8) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RW'
 elif (formationId == 8) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RDM'
 elif (formationId == 8) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 8) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 8) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LW'
 elif (formationId == 9) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 9) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 9) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 9) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 9) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 9) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'LCM'
 elif (formationId == 9) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 9) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 9) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RAM'
 elif (formationId == 9) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LAM'
 elif (formationId == 10) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 10) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 10) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 10) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 10) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 10) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 10) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'CDM'
 elif (formationId == 10) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 10) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 10) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCM'
 elif (formationId == 11) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 11) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 11) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 11) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 11) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 11) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RM'
 elif (formationId == 11) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 11) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 11) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'LCM'
 elif (formationId == 11) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LM'
 elif (formationId == 12) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 12) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 12) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 12) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 12) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 12) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RDM'
 elif (formationId == 12) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LDM'
 elif (formationId == 12) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 12) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 12) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'CAM'
 elif (formationId == 13) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 13) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 13) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 13) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 13) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 13) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 13) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 13) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 13) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RW'
 elif (formationId == 13) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LW'
 elif (formationId == 14) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RCM'
 elif (formationId == 14) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LCM'
 elif (formationId == 14) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'CDM'
 elif (formationId == 14) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 14) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 14) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'LCB'
 elif (formationId == 14) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'CCM'
 elif (formationId == 14) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'RCF'
 elif (formationId == 14) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 14) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCF'
 elif (formationId == 15) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RB'
 elif (formationId == 15) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LB'
 elif (formationId == 15) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'RDM'
 elif (formationId == 15) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'RCB'
 elif (formationId == 15) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'LCB'
 elif (formationId == 15) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'LDM'
 elif (formationId == 15) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'RCM'
 elif (formationId == 15) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LCF'
 elif (formationId == 15) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 15) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCM'
 elif (formationId == 16) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 16) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 16) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 16) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 16) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 16) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RDM'
 elif (formationId == 16) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LDM'
 elif (formationId == 16) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CF'
 elif (formationId == 16) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'CAM'
 elif (formationId == 16) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'CCM'
 elif (formationId == 17) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 17) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 17) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 17) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 17) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 17) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 17) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 17) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'LAM'
 elif (formationId == 17) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RAM'
 elif (formationId == 17) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'CF'
 elif (formationId == 18) and (formations.formationSlots[index] == 2):
  formations.position[index] = 'RWB'
 elif (formationId == 18) and (formations.formationSlots[index] == 3):
  formations.position[index] = 'LWB'
 elif (formationId == 18) and (formations.formationSlots[index] == 4):
  formations.position[index] = 'LCB'
 elif (formationId == 18) and (formations.formationSlots[index] == 5):
  formations.position[index] = 'CCB'
 elif (formationId == 18) and (formations.formationSlots[index] == 6):
  formations.position[index] = 'RCB'
 elif (formationId == 18) and (formations.formationSlots[index] == 7):
  formations.position[index] = 'RCM'
 elif (formationId == 18) and (formations.formationSlots[index] == 8):
  formations.position[index] = 'LCM'
 elif (formationId == 18) and (formations.formationSlots[index] == 9):
  formations.position[index] = 'CAM'
 elif (formationId == 18) and (formations.formationSlots[index] == 10):
  formations.position[index] = 'RCF'
 elif (formationId == 18) and (formations.formationSlots[index] == 11):
  formations.position[index] = 'LCF'


C:\Users\menes\AppData\Local\Temp\ipykernel_16932\3273369409.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formations.position[index] = 'GK'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\3273369409.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formations.position[index] = 'RB'
C:\Users\menes\AppData\Local\Temp\ipykernel_16932\3273369409.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formations.position[index] = '

In [34]:
# Merge posições

formations = formations[['playerId','position','startMinuteExpanded', 'endMinuteExpanded' ]]
formations = formations[formations.position != 'Bench']



In [35]:

merged_df = x.merge(formations, on='playerId',how='left')

In [37]:
merged_df['position_filled'] = np.where(
    (merged_df['minute'] >= merged_df['startMinuteExpanded']) &
    (merged_df['minute'] <= merged_df['endMinuteExpanded']),
    merged_df['position'],
    np.nan
)


In [39]:
#merged_df = merged_df.drop(['startMinuteExpanded', 'endMinuteExpanded','position_y','position_x'], axis=1)

In [40]:
for index, playerId in enumerate(merged_df.playerId):
 if (playerId == 0):
  merged_df.position_filled[index] = ''

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\2736134741.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.position_filled[index] = ''


In [41]:
for row, type in enumerate(merged_df.type):
    if (type == 'Goal Conceded') :
        merged_df.position_filled[row]='GK'

C:\Users\menes\AppData\Local\Temp\ipykernel_16932\3068284913.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.position_filled[row]='GK'


In [42]:
merged_df = merged_df.dropna(subset=['position_filled'])

In [43]:
merged_df = merged_df.rename(columns={'position_filled': 'position'})

In [44]:
merged_df.columns

Index(['id', 'gameWeek', 'match', 'matchId', 'homeTeamId', 'awayTeamId',
       'field', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'isTouch', 'playerId', 'endX', 'endY', 'relatedEventId',
       'relatedPlayerId', 'blockedX', 'blockedY', 'goalMouthZ', 'goalMouthY',
       'isShot', 'isGoal', 'cardType', 'timestamp', 'isGoodSkill', 'bodyPart',
       'situation', 'passType', 'isChipped', 'isBigChance', 'isKeyPass',
       'isAssist', 'isLongBall', 'is2ndAssist', 'receivedFrom', 'deliveredTo',
       'distance', 'leadingTo', 'homeTeamScore', 'awayTeamScore', 'gameState',
       'position', 'startMinuteExpanded', 'endMinuteExpanded', 'position'],
      dtype='object')

In [ ]:
# Corrigido para Dribble + is Good Skill: true, fica marado se for apagado antes
x= x[x.type !='Good Skill']

In [ ]:
#teste.to_excel("/Users/andreviegassimoes/Documents/VS Code/Jogos/teste2.xlsx")